### Task #1: Get info box (store in Python dictionary)

#### Import necessary libraries

In [4]:
from bs4 import BeautifulSoup as bs
import requests

In [12]:
r = requests.get('https://en.wikipedia.org/wiki/Inspector_Gadget_(film)')

# Convert to a beautiful soup object
soup = bs(r.content)

# Getting the HTML code
contents = soup.prettify()

In [13]:
# Finding infobox class to get table info from wiki
info_box = soup.find(class_="infobox vevent")
info_rows = info_box.find_all('tr')
  

In [17]:
def get_content_value(row_data):
  if row_data.find('li'):
    return [li.get_text(" ", strip=True).replace('\xa0', ' ') for li in row_data('li')]
  else:
    return row_data.get_text(" ", strip=True).replace('\xa0', ' ')

# Creating a set with data info
movie_info = {}
for index, row in enumerate(info_rows):
  if index == 0:
    movie_info['title'] = row.find('th').get_text(" ", strip=True)
  elif index == 1:
    continue
  else:
    content_key = row.find('th').get_text(" ", strip=True)
    content_value = get_content_value(row.find('td'))
    movie_info[content_key] = content_value

movie_info

{'title': 'Inspector Gadget',
 'Directed by': 'David Kellogg',
 'Screenplay by': ['Kerry Ehrin', 'Zak Penn', 'Audrey Wells (uncredited)'],
 'Story by': ['Kerry Ehrin', 'Dana Olsen'],
 'Based on': 'Inspector Gadget by Bruno Bianchi Jean Chalopin Andy Heyward',
 'Produced by': ['Roger Birnbaum', 'Andy Heyward', 'Jordan Kerner'],
 'Starring': ['Matthew Broderick',
  'Rupert Everett',
  'Joely Fisher',
  'Michelle Trachtenberg',
  'Mike Hagerty',
  'Andy Dick',
  'Cheri Oteri',
  'Dabney Coleman'],
 'Cinematography': 'Adam Greenberg',
 'Edited by': ['Alan Cody', 'Thom Noble'],
 'Music by': 'John Debney',
 'Production companies': ['Walt Disney Pictures [1]',
  'Caravan Pictures',
  'DIC Entertainment',
  'Avnet/Kerner Productions',
  'Roger Birnbaum Productions'],
 'Distributed by': 'Buena Vista Pictures Distribution [1]',
 'Release date': ['July 23, 1999 ( 1999-07-23 )'],
 'Running time': '78 minutes [2]',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$75–90 million [3] 